In [28]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-

import cx_Oracle
import time
import os
import datetime
import pymysql
import urllib.parse

def conndb(username, pwd, servername, ip, port,):
#定义数据库连接，需要输入三个参数
    username = username
    pwd = pwd
    tns = cx_Oracle.makedsn(ip, port, servername)
    db = cx_Oracle.connect(username,pwd,tns)
    return db

def insert_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    cursor.close()
    db.commit()

def insert_db_p(db, sql, para):
    cursor = db.cursor()
    cursor.execute(sql, para)
    cursor.close()
    db.commit()
    
def delete_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    cursor.close()
    
def select_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    result = cursor.fetchall()
    cursor.close()
    return result

def recored(path, name, state):
    #f = open(r'C:\Users\yaol\Desktop\automatic_data_input.log', 'a')
    now = time.strftime('%Y-%m-%d_%H:%M:%S',time.localtime(time.time()))
    f = open(path, 'a')
    f.write(now + ' ' + name + state + '\n')
    f.close()

# path = r'D:\DO_Automatic_data_storage\DO_搜索词入库记录.log'
path = r'C:\Users\yaol\Desktop\DO_搜索词入库记录.log'

try:
#   连接oracle
    db = conndb('fast', 'F@st*123', 'FAST', '192.156.17.197',1521)
    recored(path, 'connect to FAST database', ' =====>  Success')
#   连接mysql
    mysqlclient = pymysql.connect(host='192.153.1.154',
                                 user='root',
                                 password='10301',
                                 port = 50014,
                                 db = 'lnsy_bigdata_db',
                                 charset = "utf8")
    mysqlcursor = mysqlclient.cursor()
    recored(path, 'connect to MySQL(154) database', ' =====>  Success')
###########################################################    
    oracle_select_sql = "select to_char(sysdate - interval '1' day,'yyyymmdd') from dual"  
    row = select_db(db, oracle_select_sql)
    date_msg = '搜索词待入库数据日期：' +  row[0][0]   
    for h in range(0, 24):
        n = str(h)
        s = n.zfill(2)  # 保证输出的数字为两位
        for dblink in range(1, 3):
            t_table = row[0][0] + s + '@do_dblink' + str(dblink) + ' t'  # 重组table name 格式如：2017091921@do_dblink1 t
###########################################################
            Decode_search_word = '''select 
        to_char(ts_start,'yyyy-mm-dd-hh24') sdate,imsi,uri,uri_path,
        regexp_substr(
           regexp_substr(
              regexp_substr(uri_path,'\W(wd|query|q|word|kw|key|qw|keyword|field-keywords)=\W[A-Z]\d.+')
           ,'[^&]+',2,1)
        ,'[^(wd|query|q|word|kw|key|qw|keyword|field-keywords)=].+')  search_word,
        case when regexp_substr(uri_path,'=gbk') = '=gbk' then 1 
          when regexp_substr(uri_path,'=gbk2312') = '=gbk2312' then 2 else 0 end encode
    from s1dss2_{}
    where (URI_PATH like '%wd=%' or uri_path like '%query=%' or uri_path like '%q=%' or uri_path like '%word=%' or uri_path like '%kw=%' or 
    uri_path like '%key=%' or uri_path like '%wq=%' or uri_path like '%keyword=%' or uri_path like '%field-keywords=%') and imsi > 0 and 
    regexp_substr(
           regexp_substr(
              regexp_substr(uri_path,'\W(wd|query|q|word|kw|key|qw|keyword|field-keywords)=\W[A-Z]\d.+')
           ,'[^&]+',2,1)
        ,'[^(wd|query|q|word|kw|key|qw|keyword|field-keywords)=].+')  is not null'''.format(t_table)
###########################################################    

            try:
                print("执行 =====>  ", 's1dss2_' + t_table)
                s1dss2 = 's1dss2_' + t_table
                r = select_db(db, Decode_search_word)
                for i in r:
                    decode = urllib.parse.unquote(i[4])
                    mysql_insert_sql = '''insert into aaa(sdate,imsi,uri,uri_path,search_word,encode,decode_search_word) 
                    values('{}','{}','{}','{}','{}',{},'{}')'''.format(i[0],i[1],i[2],i[3],i[4],i[5],decode)
                    try:
                        mysqlcursor = client.cursor()
                        mysqlcursor.execute(mysql_insert_sql)
                        client.commit()
                    except pymysql.DatabaseError as mysqlmsg:
                        recored(path,'插入失败', ' =====>  Fail  Error msg: {}'.format(mysqlmsg))             
                recored(path, s1dss2, ' =====>  Success')

            except cx_Oracle.DatabaseError as msg:
                s1dss2 = 's1dss2_' + t_table
                print('s1dss2', ' =====>  Fail  Error msg: {}'.format(msg))
except:
    recored(path, 'connect to fast database', ' =====>  Fail')
recored(path, 'The program has been executed!', ' ')



执行 =====>   s1dss2_2018020100@do_dblink1 t
执行 =====>   s1dss2_2018020100@do_dblink2 t
s1dss2  =====>  Fail  Error msg: ORA-28001: the password has expired
ORA-02063: preceding line from DO_DBLINK2
执行 =====>   s1dss2_2018020101@do_dblink1 t
执行 =====>   s1dss2_2018020101@do_dblink2 t
s1dss2  =====>  Fail  Error msg: ORA-28001: the password has expired
ORA-02063: preceding line from DO_DBLINK2
执行 =====>   s1dss2_2018020102@do_dblink1 t
执行 =====>   s1dss2_2018020102@do_dblink2 t
s1dss2  =====>  Fail  Error msg: ORA-28001: the password has expired
ORA-02063: preceding line from DO_DBLINK2
执行 =====>   s1dss2_2018020103@do_dblink1 t
执行 =====>   s1dss2_2018020103@do_dblink2 t
s1dss2  =====>  Fail  Error msg: ORA-28001: the password has expired
ORA-02063: preceding line from DO_DBLINK2
执行 =====>   s1dss2_2018020104@do_dblink1 t
执行 =====>   s1dss2_2018020104@do_dblink2 t
s1dss2  =====>  Fail  Error msg: ORA-28001: the password has expired
ORA-02063: preceding line from DO_DBLINK2
执行 =====>   s1d